# Repopulation Paris

## First Look into Bottin Data

In [1]:
import pandas as pd

In [2]:
bottins = pd.read_csv("data/strict_addressing.csv")

In [3]:
print("size of table:", bottins.shape)
bottins.head(10)

size of table: (4406164, 8)


,Unnamed: 0,page,row,Nom,Métier,Rue,Numéro,annee
0,bpt6k6282019m,144,0,Aaron,bronzes,passage Choiseal,72 et 74.,1855
1,bpt6k6282019m,144,1,Aaron (Mic.),manuf. de porcelaines,Bondy,30.,1855
2,bpt6k6282019m,144,3,Abadie,architecte,Provence,7.,1855
3,bpt6k6282019m,144,5,Abadie,tabac et estamin.,Ménilmontant,158.,1855
4,bpt6k6282019m,144,6,Abanse,instituteur,Sts-Pères,30.,1855
5,bpt6k6282019m,144,7,Abat,rentier,Isly,10.,1855
6,bpt6k6282019m,144,13,Abault et Coudray,charpentiers,Corbeau,23.,1855
7,bpt6k6282019m,144,14,Abault (Paul),libraire,quai des Angustins,9.,1855
8,bpt6k6282019m,144,15,Abavid,vins,Beaujolais-da-Temple,7.,1855
9,bpt6k6282019m,144,16,Abazaer (Are),cristaux et porcelaines,Pei.Ecuries,26.,1855


In [4]:
print("number of streets in dataset:", len(bottins["Rue"].unique()))

number of streets in dataset: 195464


In [5]:
bottins_original = bottins.copy()

## Paris Opendata - Voies

In [6]:
voies = pd.read_csv("data/opendata_voie_paris.csv", sep=";")

In [7]:
print("number of voies:", len(voies))
voies.head()

number of voies: 6542


,N_SQ_VO,C_COINSEE,C_DESI,C_LIAISON,L_VOIE,L_COURTMIN,L_LONGMIN,C_VOIE,B_FANTOIR,B_OFF,C_VOIE_VP,Geometry,OBJECTID,C_DOMOFF,N_SQ_CO,LENGTH,Geometry X Y
0,750004016,75056,IMP,NaN,BAUDRAN,Imp. Baudran,Impasse Baudran,750560733,O,O,0717,"{""coordinates"": [[2.3554518641916804, 48.82415...",6,O,750001537,64.625200,"48.82415332715383,2.3558919735528483"
1,750003821,75056,IMP,DES,CHEVALIERS,Imp. des Chevaliers,Impasse des Chevaliers,750561994,O,O,1986,"{""coordinates"": [[2.3960916096233094, 48.87258...",59,P,750001537,77.423197,"48.87229377669366,2.3963259349268453"
2,750005946,75056,RUE,DES,BONS VIVANTS,R. des Bons Vivants,Rue des Bons Vivants,750561106,O,O,1111,"{""coordinates"": [[2.346752283163513, 48.862212...",112,P,750001537,39.406151,"48.862044141078364,2.34682583693024"
3,750001312,75056,RUE,NaN,JEAN HUGUES,R. Jean Hugues,Rue Jean Hugues,750564899,O,O,4806,"{""coordinates"": [[2.274705254077503, 48.868429...",170,O,750001537,45.962263,"48.86861817275934,2.2748341164181163"
4,750005774,75056,RUE,NaN,JEAN-JACQUES ROUSSEAU,R. Jean-Jacques Rousseau,Rue Jean-Jacques Rousseau,750564901,O,O,4808,"{""coordinates"": [[[2.3396869374353004, 48.8620...",171,P,750001537,397.842033,"48.86347293952501,2.342392728552715"


In [8]:
#keep only columns that might be useful further on
voies = voies[["N_SQ_VO", "L_VOIE", "L_COURTMIN", "L_LONGMIN", "Geometry", "LENGTH", "Geometry X Y"]]
voies.head()

,N_SQ_VO,L_VOIE,L_COURTMIN,L_LONGMIN,Geometry,LENGTH,Geometry X Y
0,750004016,BAUDRAN,Imp. Baudran,Impasse Baudran,"{""coordinates"": [[2.3554518641916804, 48.82415...",64.625200,"48.82415332715383,2.3558919735528483"
1,750003821,CHEVALIERS,Imp. des Chevaliers,Impasse des Chevaliers,"{""coordinates"": [[2.3960916096233094, 48.87258...",77.423197,"48.87229377669366,2.3963259349268453"
2,750005946,BONS VIVANTS,R. des Bons Vivants,Rue des Bons Vivants,"{""coordinates"": [[2.346752283163513, 48.862212...",39.406151,"48.862044141078364,2.34682583693024"
3,750001312,JEAN HUGUES,R. Jean Hugues,Rue Jean Hugues,"{""coordinates"": [[2.274705254077503, 48.868429...",45.962263,"48.86861817275934,2.2748341164181163"
4,750005774,JEAN-JACQUES ROUSSEAU,R. Jean-Jacques Rousseau,Rue Jean-Jacques Rousseau,"{""coordinates"": [[[2.3396869374353004, 48.8620...",397.842033,"48.86347293952501,2.342392728552715"


In [9]:
# prepare data for alignment
bottins["Rue"] = bottins["Rue"].str.lower()
voies["L_VOIE"] = voies["L_VOIE"].str.lower()
voies["L_COURTMIN"] = voies["L_COURTMIN"].str.lower()
voies["L_LONGMIN"] = voies["L_LONGMIN"].str.lower()

In [10]:
bot_voies_algn1 = bottins.join(voies.set_index("L_LONGMIN"), on="Rue")
bot_voies_algn2 = bottins.join(voies.set_index("L_VOIE"), on="Rue")
bot_voies_algn3 = bottins.join(voies.set_index("L_COURTMIN"), on="Rue")

In [11]:
# see how many streets were aligned
print(bot_voies_algn1["Geometry"].isna().sum())
print(bot_voies_algn2["Geometry"].isna().sum())
print(bot_voies_algn3["Geometry"].isna().sum())

4228735
3191199
3793594


In [12]:
# start with 2nd dataframe as basis, split into aligned an not aligned data
aligned = bot_voies_algn2.loc[bot_voies_algn2["Geometry"].notna()]
not_aligned = bot_voies_algn2.loc[bot_voies_algn2["Geometry"].isna()]
not_aligned = not_aligned.drop(["L_COURTMIN", "L_LONGMIN", "Geometry", "LENGTH", "Geometry X Y"], axis=1)
print("aligned:", len(aligned), "not aligned:", len(not_aligned))

aligned: 1933426 not aligned: 3191199


In [13]:
# perform join with "L_COURTMIN on not_aligned dataframe"
x = not_aligned.join(voies.set_index("L_COURTMIN"), on="Rue")
aligned = aligned.append(x[x["Geometry"].notna()], ignore_index = True)
not_aligned = x[x["Geometry"].isna()]
not_aligned = not_aligned.drop(["L_LONGMIN", "L_VOIE", "Geometry", "LENGTH", "Geometry X Y"], axis=1)
print("aligned:", len(aligned), "not aligned:", len(not_aligned))

ValueError: columns overlap but no suffix specified: Index(['N_SQ_VO'], dtype='object')

In [ ]:
# perform join with "L_LONGMIN on not_aligned dataframe"
x = not_aligned.join(voies.set_index("L_LONGMIN"), on="Rue")
aligned = aligned.append(x[x["Geometry"].notna()], ignore_index = True)
not_aligned = x[x["Geometry"].isna()]
print("aligned:", len(aligned), "not aligned:", len(not_aligned))

In [ ]:
not_aligned.tail()

In [ ]:
# Count how many addresses were aligned per year
annee_aligned = aligned.groupby(by=["annee"]).count()[["Nom"]]
annee_not_aligned = not_aligned.groupby(by=["annee"]).count()[["Nom"]]
df_per_year = annee_aligned.join(annee_not_aligned, lsuffix='_aligned', rsuffix='_not_aligned')

In [ ]:
# Calculate the ratio of aligned addresses
df_per_year["ratio_aligned"] = df_per_year["Nom_aligned"] / (df_per_year["Nom_aligned"]+df_per_year["Nom_not_aligned"])
df_per_year

## Paris Opendata - Adresses

In [ ]:
adresses = pd.read_csv("data/opendata_adresse_paris.csv", sep=";")

In [ ]:
adresses.head()
# N_SQ_VO same as with voies! 

In [ ]:
adresses[["Numéro", "Rue"]] = adresses["L_ADR"].str.split(" ", 1, expand=True)
adresses["Rue"] = adresses["Rue"].str.lower()
adresses.head()